In [2]:
!pip install beautifulsoup4
!pip install requests
!pip install re # trabalhar com regex

ERROR: Invalid requirement: '#'


Para esse estudo vamos extrair informações dos candidatos  a prefeitos de 3 cidades.
Captura será feita pelo site: "https://politica.estadao.com.br/eleicoes/2020/candidatos".

Seguites informações serão coletadas:

1. Nome do Candidato;
2. Partido do Candidato;
3. O Estado;
4. A Cidade;
5. O Endereço de link do candidato.

In [3]:
#importando as bibliotecas

import requests
from bs4 import BeautifulSoup
import re

In [4]:
# Criação da requisição ao link que contém as informações.

# Criação da variavel e atribuição.
r = requests.get("https://politica.estadao.com.br/eleicoes/2020/candidatos")

In [8]:
# Visualizar o conteúdo do texto

r#.text

<Response [200]>

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')

In [7]:
type(soup)

bs4.BeautifulSoup

In [9]:
candidatos = soup.find_all("div", attrs={"class": "col-xs-12 -only"})

In [10]:
candidatos[:3]

[<div class="col-xs-12 -only"> <a href="https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/andrea-matarazzo,55" title="andrea matarazzo"> <div class="mold-cand"> <div class="cand-mask"> <img alt="andrea matarazzo" class="elazy" data-src="https://img.estadao.com.br/fotos/politica/eleicoes-2020/SP/FSP250000661535_div.jpg" onerror="this.onerror=null;this.src='https://statics.estadao.com.br/s2016/portal/eleicoes2020/img/img_placeholder.png';"/> </div> <div class="cand-desc"> <h3 class="cand-name">andrea matarazzo</h3> <span class="cand-partido">PSD</span> </div> </div> </a> </div>,
 <div class="col-xs-12 -only"> <a href="https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/antonio-carlos,29" title="antônio carlos"> <div class="mold-cand"> <div class="cand-mask"> <img alt="antônio carlos" class="elazy" data-src="https://img.estadao.com.br/fotos/politica/eleicoes-2020/SP/FSP250001172314_div.jpg" onerror="this.onerror=null;this.src='https:

In [12]:
# Extração do nome candidato utilizando a tag "img"
nome_candidato = candidatos[1].img["alt"] # Traz o primeiro elemento da lista
# Extração do nome candidatos utilizando a TAG "a"
nome_candidato2 = candidatos[1].a["title"]
print('Extração do nome candidato utilizando a TAG "img"',nome_candidato)
print('Extração do nome candidatos utilizando a TAG "a"', nome_candidato2)

Extração do nome candidato utilizando a TAG "img" antônio carlos
Extração do nome candidatos utilizando a TAG "a" antônio carlos


In [13]:
# Buscando partido do candidato
partido = candidatos[1].span.text
print(partido)

PCO


A informação de estadp e cidade está dentro da TAG "a". A TAG "a" é um link de acesso, então vamos utilizar expressão regular para coletar essa informação pelo link

In [14]:
import re

In [15]:
print(candidatos[1].a['href'])

https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/antonio-carlos,29


In [16]:
print(re.split('/', candidatos[1].a['href']))

['https:', '', 'politica.estadao.com.br', 'eleicoes', '2020', 'candidatos', 'sp', 'sao-paulo', 'prefeito', 'antonio-carlos,29']


In [19]:
print("Estado do Candidato:", re.split('/', candidatos[1].a['href'])[6])
print("cidade do Candidato:", re.split('/', candidatos[1].a['href'])[7])

Estado do Candidato: sp
cidade do Candidato: sao-paulo


In [21]:
estado = re.split('/', candidatos[1].a['href'])[6]
cidade = re.split('/', candidatos[1].a['href'])[7]
link = candidatos[1].a["href"]

In [22]:
print(estado, cidade, link)

sp sao-paulo https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/antonio-carlos,29


In [44]:
lista_candidatos = []
# busca os nomes de todos os candidatos
for candidato in candidatos:
    dados_candidato = []
    dados_candidato.append(candidato.img["alt"]) # nome do candidato
    dados_candidato.append(candidato.span.text) # Partido
    dados_candidato.append(re.split('/', candidato.a['href'])[6]) # Estado
    dados_candidato.append(re.split('/', candidato.a['href'])[7]) # Cidade
    dados_candidato.append(candidato.a['href']) # link
    lista_candidatos.append(dados_candidato)

In [26]:
lista_candidatos[:2]

[['andrea matarazzo',
  'PSD',
  'sp',
  'sao-paulo',
  'https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/andrea-matarazzo,55'],
 ['antônio carlos',
  'PCO',
  'sp',
  'sao-paulo',
  'https://politica.estadao.com.br/eleicoes/2020/candidatos/sp/sao-paulo/prefeito/antonio-carlos,29']]

In [27]:
import pandas as pd

In [29]:
df_candidatos = pd.DataFrame(lista_candidatos, columns=['nome_candidato', 'partido', 'estado', 'cidade', 'link'])

In [31]:
df_candidatos[:4]

,nome_candidato,partido,estado,cidade,link
0,andrea matarazzo,PSD,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
1,antônio carlos,PCO,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
2,arthur do val mamãe falei,PATRIOTA,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
3,bruno covas,PSDB,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...


### Explorando e tratando os dados coletados

In [32]:
df_candidatos['estado'].value_counts()

mg    16
sp    14
rj    14
Name: estado, dtype: int64

In [33]:
df_candidatos['cidade'].value_counts()

belo-horizonte    16
sao-paulo         14
rio-de-janeiro    14
Name: cidade, dtype: int64

In [34]:
df_candidatos['partido'].value_counts()

PCO              3
REPUBLICANOS     3
PSOL             3
PT               3
PSTU             3
PRTB             3
NOVO             3
PSD              2
PMB              2
PROS             2
PC do B          2
REDE             2
PSL              2
PSDB             2
PATRIOTA         2
PSB              1
PDT              1
DEM              1
PSC              1
MDB              1
CIDADANIA        1
SOLIDARIEDADE    1
Name: partido, dtype: int64

### Realizando tratamento de texto

In [35]:
df_candidatos[:5]

,nome_candidato,partido,estado,cidade,link
0,andrea matarazzo,PSD,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
1,antônio carlos,PCO,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
2,arthur do val mamãe falei,PATRIOTA,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
3,bruno covas,PSDB,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...
4,celso russomanno,REPUBLICANOS,sp,sao-paulo,https://politica.estadao.com.br/eleicoes/2020/...


In [36]:
df_candidatos[:2].cidade.str.replace('-', ' ').str.upper()

0    SAO PAULO
1    SAO PAULO
Name: cidade, dtype: object

In [37]:
df_candidatos[:2].cidade.str.replace('-', ' ').str.upper().str.replace("SAO", "SÃO")

0    SÃO PAULO
1    SÃO PAULO
Name: cidade, dtype: object

In [38]:
df_candidatos['cidade'] = df_candidatos.cidade.str.replace('-', ' ').str.upper().str.replace("SAO", "SÃO")

In [40]:
df_candidatos[:3]

,nome_candidato,partido,estado,cidade,link
0,andrea matarazzo,PSD,sp,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
1,antônio carlos,PCO,sp,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
2,arthur do val mamãe falei,PATRIOTA,sp,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...


In [45]:
# Transformar os campos do nome do candidato e estado em maiúsculo

df_candidatos.nome_candidato = df_candidatos.nome_candidato.str.upper()
df_candidatos.estado = df_candidatos.estado.str.upper()

# Visualizar Data Frame
df_candidatos[:3]

,nome_candidato,partido,estado,cidade,link
0,ANDREA MATARAZZO,PSD,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
1,ANTÔNIO CARLOS,PCO,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
2,ARTHUR DO VAL MAMÃE FALEI,PATRIOTA,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...


# Verificando coleta de dados

In [46]:
df_candidatos

,nome_candidato,partido,estado,cidade,link
0,ANDREA MATARAZZO,PSD,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
1,ANTÔNIO CARLOS,PCO,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
2,ARTHUR DO VAL MAMÃE FALEI,PATRIOTA,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
3,BRUNO COVAS,PSDB,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
4,CELSO RUSSOMANNO,REPUBLICANOS,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
5,GUILHERME BOULOS,PSOL,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
6,JILMAR TATTO,PT,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
7,JOICE HASSELMANN,PSL,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
8,LEVY FIDELIX,PRTB,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
9,MARINA HELOU,REDE,SP,SÃO PAULO,https://politica.estadao.com.br/eleicoes/2020/...
